In [1]:
import sys
sys.path.append("..")
from utils.audio_utils import get_total_audio_duration,get_audio_info
from pathlib import Path
from tqdm import tqdm
from utils.file_utils import save_json
from utils.transcription_utils import clean_and_verify_transcript_hi,get_transcription_info,get_stats,get_total_stats
from collections import Counter
from utils.conversions import sec_to_time
import csv

In [2]:
save_path=Path("/media/dataset-harddisk/munikumar/utils/audio_utils/data/common_voice/nuktas")

In [3]:
dataset_dir=Path("/media/dataset-harddisk/munikumar/hindi_dataset/common_voice/cv-corpus-13.0-2023-03-09/hi")

In [4]:
def load_transcription(transcription_path):
    with open(transcription_path, "r", encoding="utf-8") as fp:
        reader=csv.reader(fp,delimiter='\t')
        next(reader)
        transcription_data = {
            transcription[1]:transcription[2]
            for transcription in reader
        }
    return transcription_data

In [5]:
test_transcription_data=load_transcription(dataset_dir/"test.tsv")
get_transcription_info(test_transcription_data,dataset_dir/"clips",save_path,"test")
get_stats(save_path/"test_non_hindi_transcripts.json")

getting audio info about : nuktas - test


  1%|          | 17/2947 [00:01<03:11, 15.27it/s]

100%|██████████| 2947/2947 [02:33<00:00, 19.15it/s]


**************************************************
Loss of audio data without transcription normalization:
**************************************************
Total duration : 4:4:19
Loss duration : 3:14:12
Remaining duration : 0:50:6
Drop by percentage: 79.49

Total audio files : 2947
Files that would be drop: 2396
Remaining audio files : 551
Drop by percentage: 81.30
**************************************************
Non hindi tokes : ['!', '"', '&', "'", ',', '-', '.', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'J', 'K', 'L', 'M', 'O', 'P', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '।', '–', '‘', '’']


### Test Data
**************************************************
**Loss of audio data without transcription normalization:**
**************************************************
Total duration : 4:4:19  
Loss duration : 3:14:59  
Remaining duration : 0:49:19  
Drop by percentage: 79.81  

Total audio files : 2947  
Files that would be drop: 2404  
Remaining audio files : 543  
Drop by percentage: 81.57  
**************************************************
Non hindi tokes : ['!', '"', '&', "'", ',', '-', '.', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'J', 'K', 'L', 'M', 'O', 'P', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', 'ख़', 'ज़', 'ड़', 'ढ़', '।', '–', '‘', '’']

**************************************************
**Loss of audio data After Decomposing Nuktas:**
**************************************************
Total duration : 4:4:19  
Loss duration : 3:14:12  
Remaining duration : 0:50:6  
Drop by percentage: 79.49  

Total audio files : 2947  
Files that would be drop: 2396  
Remaining audio files : 551  
Drop by percentage: 81.30  
**************************************************
Non hindi tokes : ['!', '"', '&', "'", ',', '-', '.', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'J', 'K', 'L', 'M', 'O', 'P', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '।', '–', '‘', '’']

In [6]:
dev_transcription_data=load_transcription(dataset_dir/"dev.tsv")
get_transcription_info(dev_transcription_data,dataset_dir/"clips",save_path,"dev")
get_stats(save_path/"dev_non_hindi_transcripts.json")

getting audio info about : nuktas - dev


  0%|          | 5/2281 [00:00<01:54, 19.84it/s]

100%|██████████| 2281/2281 [01:59<00:00, 19.12it/s]


**************************************************
Loss of audio data without transcription normalization:
**************************************************
Total duration : 2:56:15
Loss duration : 2:19:10
Remaining duration : 0:37:4
Drop by percentage: 78.96

Total audio files : 2281
Files that would be drop: 1849
Remaining audio files : 432
Drop by percentage: 81.06
**************************************************
Non hindi tokes : ['!', '"', "'", ',', '-', '.', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '।', '‘', '’']


### Dev Dataset
**************************************************
Loss of audio data without transcription normalization:
**************************************************
Total duration : 2:56:15  
Loss duration : 2:19:57  
Remaining duration : 0:36:17  
Drop by percentage: 79.41  

Total audio files : 2281  
Files that would be drop: 1858  
Remaining audio files : 423  
Drop by percentage: 81.46  
**************************************************
Non hindi tokes : ['!', '"', "'", ',', '-', '.', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', 'ग़', 'ज़', 'ड़', 'ढ़', '।', '‘',]

**************************************************
**Loss of audio data After Decomposing Nuktas:**
**************************************************
Total duration : 2:56:15  
Loss duration : 2:19:10  
Remaining duration : 0:37:4  
Drop by percentage: 78.96  

Total audio files : 2281  
Files that would be drop: 1849  
Remaining audio files : 432  
Drop by percentage: 81.06  
**************************************************
Non hindi tokes : ['!', '"', "'", ',', '-', '.', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '।', '‘', '’']

In [7]:
train_transcription_data=load_transcription(dataset_dir/"train.tsv")
get_transcription_info(train_transcription_data,dataset_dir/"clips",save_path,"train")
get_stats(save_path/"train_non_hindi_transcripts.json")

getting audio info about : nuktas - train


  0%|          | 8/4479 [00:00<03:38, 20.43it/s]

100%|██████████| 4479/4479 [03:46<00:00, 19.79it/s]


**************************************************
Loss of audio data without transcription normalization:
**************************************************
Total duration : 5:17:19
Loss duration : 3:58:50
Remaining duration : 1:18:29
Drop by percentage: 75.26

Total audio files : 4479
Files that would be drop: 3392
Remaining audio files : 1087
Drop by percentage: 75.73
**************************************************
Non hindi tokes : ['!', '"', '&', "'", ',', '-', '.', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '।', '‘', '’', '“', '”']


### Train Dataset
**************************************************
Loss of audio data without transcription normalization:
**************************************************
Total duration : 5:17:19  
Loss duration : 4:0:41  
Remaining duration : 1:16:38  
Drop by percentage: 75.85  

Total audio files : 4479  
Files that would be drop: 3418  
Remaining audio files : 1061  
Drop by percentage: 76.31  
**************************************************
Non hindi tokes : ['!', '"', '&', "'", ',', '-', '.', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', 'क़', 'ज़', 'ड़', 'ढ़', 'फ़', '।', '‘', '’', '“', '”']

**************************************************
**Loss of audio data After Decomposing Nuktas:**
**************************************************
Total duration : 5:17:19  
Loss duration : 3:58:50  
Remaining duration : 1:18:29  
Drop by percentage: 75.26  

Total audio files : 4479  
Files that would be drop: 3392    
Remaining audio files : 1087  
Drop by percentage: 75.73  
**************************************************
Non hindi tokes : ['!', '"', '&', "'", ',', '-', '.', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '।', '‘', '’', '“', '”']

In [9]:
get_total_stats(save_path)

**************************************************
Loss of audio data for total dataset:
**************************************************
Total duration : 12:17:54
Loss duration : 9:32:13
Remaining duration : 2:45:40
Drop by percentage: 77.55

Total audio files : 9707
Files that would be drop: 7637
Remaining audio files : 2070
Drop by percentage: 78.68


### Total Dataset
**************************************************
Loss of audio data for total dataset:
**************************************************
Total duration : 12:17:54  
Loss duration : 9:35:38  
Remaining duration : 2:42:15  
Drop by percentage: 78.01  

Total audio files : 9707  
Files that would be drop: 7680  
Remaining audio files : 2027  
Drop by percentage: 79.12  

**************************************************
Loss of audio data for total dataset After Decomposing Nuktas:
**************************************************
Total duration : 12:17:54  
Loss duration : 9:32:13  
Remaining duration : 2:45:40  
Drop by percentage: 77.55  

Total audio files : 9707  
Files that would be drop: 7637  
Remaining audio files : 2070  
Drop by percentage: 78.68  